# 以固定间隔枚举目标空间采样

In [1]:
from pathlib import Path
import sys
import gymnasium as gym
import panda_gym
import numpy as np
import pandas as pd
from gymnasium.envs.registration import register

PROJECT_ROOT_DIR = Path().absolute().parent
PROJECT_ROOT_DIR

if str(PROJECT_ROOT_DIR.absolute()) not in sys.path:
    sys.path.append(str(PROJECT_ROOT_DIR.absolute()))

from rollout.rollout_by_pid import rollout_by_goal_with_pid
from my_reach_env import MyPandaReachEnv


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group
/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group
/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/p

In [2]:
goal_range = 0.3
distance_threshold = 0.01

In [3]:
register(
    id="my-reach",
    entry_point=f"my_reach_env:MyPandaReachEnv",
    kwargs={"reward_type": "sparse", "control_type": "ee", "goal_range": goal_range, "distance_threshold": distance_threshold},
    max_episode_steps=100,
)

In [4]:
env = gym.make("my-reach")
env.observation_space

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


Dict('achieved_goal': Box(-10.0, 10.0, (3,), float32), 'desired_goal': Box(-10.0, 10.0, (3,), float32), 'observation': Box(-10.0, 10.0, (6,), float32))

In [5]:
env.task.goal_range_low, env.task.goal_range_high

/home/hs/anaconda3/envs/o2o/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.task` for environment variables or `env.get_wrapper_attr('task')` that will search the reminding wrappers.
  logger.warn(


(array([-0.15, -0.15,  0.  ]), array([0.15, 0.15, 0.3 ]))

In [6]:
env.task.distance_threshold

0.01

In [7]:
success_cnt = 0
total_df = None
goal_range_int = int(goal_range * 100)
speed = 1.5
x_low, x_high = -int(goal_range_int/2), int(goal_range_int/2)
y_low, y_high = -int(goal_range_int/2), int(goal_range_int/2)
z_low, z_high = 0, int(goal_range_int)
for x in range(x_low, x_high+1, 2):
    for y in range(y_low, y_high+1, 2):
        for z in range(z_low, z_high+1, 2):
            terminated, truncated, traj = rollout_by_goal_with_pid(env=env, goal=np.array([x/100., y/100., z/100.]), speed=speed)
            if terminated:
                if total_df is None:
                    total_df = pd.DataFrame(traj)
                else:
                    total_df = pd.concat([total_df, pd.DataFrame(traj)])
                success_cnt += 1

success_cnt

 success. goal: (-0.15, -0.15, 0.0), steps: 57 
 success. goal: (-0.15, -0.15, 0.02), steps: 55 
 success. goal: (-0.15, -0.15, 0.04), steps: 55 
 success. goal: (-0.15, -0.15, 0.06), steps: 54 
 success. goal: (-0.15, -0.15, 0.08), steps: 54 
 success. goal: (-0.15, -0.15, 0.1), steps: 54 
 success. goal: (-0.15, -0.15, 0.12), steps: 53 
 success. goal: (-0.15, -0.15, 0.14), steps: 53 
 success. goal: (-0.15, -0.15, 0.16), steps: 53 
 success. goal: (-0.15, -0.15, 0.18), steps: 53 
 success. goal: (-0.15, -0.15, 0.2), steps: 53 
 success. goal: (-0.15, -0.15, 0.22), steps: 53 
 success. goal: (-0.15, -0.15, 0.24), steps: 53 
 success. goal: (-0.15, -0.15, 0.26), steps: 53 
 success. goal: (-0.15, -0.15, 0.28), steps: 53 
 success. goal: (-0.15, -0.15, 0.3), steps: 53 
 success. goal: (-0.15, -0.13, 0.0), steps: 57 
 success. goal: (-0.15, -0.13, 0.02), steps: 55 
 success. goal: (-0.15, -0.13, 0.04), steps: 55 
 success. goal: (-0.15, -0.13, 0.06), steps: 54 
 success. goal: (-0.15, -

4090

In [8]:
import os
os.makedirs("cache", exist_ok=True)
total_df.to_csv(f"cache/myreach_pid_speed_{speed:.1f}.csv", index=False)

In [9]:
total_df.shape

(199956, 12)